In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.13.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\renan\OneDrive\Documentos\Insper\alglin\alglin\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


# Parte 3

# Bases: mudando o referencial

Este capítulo está ligado ao seguintes objetivos didáticos do curos:
1. Avaliar e resolver sistemas de equações lineares
1. Interpretar e analisar transformações lineares

Referência bibliográfica: [Jim Hefferon - Linear Algebra - 4th Edition](https://hefferon.net/linearalgebra/) - Chap. Three, V-VI

# Exercício 1
**Objetivo: realizar uma mudança de referencial em tempo real**

Em algumas situações, como jogos digitais, gostaríamos de mudar o referencial sobre o qual estamos visualizando. Um exemplo disso é um jogo em primeira pessoa: embora o personagem, em relação ao mundo, possa caminhar por um espaço qualquer, toda a visão que o jogador tem é de acordo com o referencial do próprio personagem.

Neste exercício, vamos partir do pequeno jogo abaixo. Ao executar o código, você deve ver um personagem verde girando no meio de vários pontos vermelhos. Veja no código que o jogador é representado pela variável `s` e os pontos ao seu redor são representados no vetor `pontos_loc`.

<img src="transforma_girando.gif" width=200>

Veja que a posição do jogador está no vetor `s`, que é recalculado a cada iteração do jogo.

1. Gostaríamos de manter nosso jogador sempre no centro da tela. Através de uma nova matriz $T$ (crie essa matriz!), transforme os pontos do vetor `pontos_loc` e do próprio vetor `s` de forma que a tela fique sempre centralizada no jogador. Seu resultado deveria ser um ponto verde parado na tela e vários pontos vermelhos girando ao redor dele. Lembre-se que o ponto central da tela é o ponto $(200,200)$ O resultado deve ser parecido com isso:

<img src="transforma_translacao.gif" width=200>

2. Após, gostaríamos de manter a "frente" do jogador sempre virada para cima, isto é, vamos fazer uma transformação para que todos os pontos do mundo virtual girem em torno do nosso personagem. O resultado deve ser algo parecido com:

<img src="transforma_rotacao.gif" width=200>

Dicas:

(a)
Uma matriz de *translação* se parece com:

$$
T = \begin{bmatrix}
1 & 0 & \Delta x \\
0 & 1 & \Delta y \\
0 & 0 & 1 
\end{bmatrix}
$$

(b)
Uma matriz de *rotação*, que realiza a rotação de vetores *ao redor da origem $[0,0]$*, se parece com:
$$
T = \begin{bmatrix}
\cos(\theta) & -\sin(\theta) & 0 \\
\sin(\theta) & \cos(\theta) & 0 \\
0 & 0 & 1 
\end{bmatrix}
$$

(c)
Se uma curva é parametrizada por $v = [f_x(t), f_y(t)]$, então o vetor tangente a ela é $v' = [f'_x(t), f'_y(t)]$.

(d)
O menor ângulo entre um vetor e o eixo horizontal é chamado argumento e pode ser calculado usando `np.arctan2`.

#### Parte 1 - Jogador travado no centro da tela

In [4]:
import pygame
import numpy as np
pygame.init()

# Tenho aqui vários pontos sobre uma circunferência
angulo = np.linspace(0, 2*np.pi, 20)
pontos_loc = np.vstack ( (100*np.cos(angulo)+200, 100*np.sin(angulo)+200, np.ones(20)) )

# Controle de tempo
t = 0

# Velocidade angular (rotacoes por segundo)
v = 0.2

# Tamanho da tela e definição do FPS
screen = pygame.display.set_mode((400, 400))
clock = pygame.time.Clock()
FPS = 60  # Frames per Second

BLACK = (0, 0, 0)
COR_PERSONAGEM = (30, 200, 20)
COR_PONTOS = (200, 30, 20)

# Personagem
personagem = pygame.Surface((5, 5))  # Tamanho do personagem
personagem.fill(COR_PERSONAGEM)  # Cor do personagem

# Pontos 
pontos = pygame.Surface((5, 5))
pontos.fill(COR_PONTOS)  # Cor dos pontos

rodando = True
while rodando:
    # Capturar eventos
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            rodando = False

    # Controlar frame rate
    clock.tick(FPS)

    # Movimento do personagem
    t += 1/FPS
    theta = v*t
    s = np.array([[200],[200]]) + 100 * np.array([ [np.cos(2*np.pi*theta)], [-np.sin(2*np.pi*theta)]])
    s = np.vstack( (s, np.ones(1)))

    # MODIFICAÇÕES A PARTIR DAQUI

    # Delta X e Delta Y
    # s[0,0] é a coordenada x do personagem
    # s[1,0] é a coordenada y do personagem
    dx = s[0,0] - 200
    dy = s[1,0] - 200

    # Matriz de transformação
    # Usamos dx e dy para "mover" os pontos ao redor do personagem
    T = np.array([[1, 0, dx],
                  [0, 1, dy],
                  [0, 0, 1]])

    # Pontos na nova posição
    pontos_loc_y = pontos_loc
    pontos_loc_y = T @ pontos_loc_y

    # Desenhar fundo
    screen.fill(BLACK)

    # Desenhar pontos
    for p in range(pontos_loc.shape[1]):
        rect = pygame.Rect(pontos_loc_y[0:2,p], (2, 2))  # First tuple is position, second is size.
        screen.blit(pontos, rect)
        
    # Desenhar personagem (no centro da tela)
    rect = pygame.Rect([200,200], (10, 10))  # First tuple is position, second is size.
    screen.blit(personagem, rect)

    # Update!
    pygame.display.update()

# Terminar tela
pygame.quit()

#### Parte 2 - Fixando um centro para os pontos girarem ao redor do jogador

In [ ]:
import pygame
import numpy as np
pygame.init()

# Pontos sobre uma circunferência
angulo = np.linspace(0, 2*np.pi, 20)
pontos_loc = np.vstack((100*np.cos(angulo)+200, 100*np.sin(angulo)+200, np.ones(20)))

# Controle de tempo
t = 0
v = 0.2

screen = pygame.display.set_mode((400, 400))
clock = pygame.time.Clock()
FPS = 60

BLACK = (0, 0, 0)
COR_PERSONAGEM = (30, 200, 20)
COR_PONTOS = (200, 30, 20)

personagem = pygame.Surface((5, 5))
personagem.fill(COR_PERSONAGEM)
pontos = pygame.Surface((5, 5))
pontos.fill(COR_PONTOS)

rodando = True
while rodando:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            rodando = False

    clock.tick(FPS)
    t += 1/FPS
    theta = v*t

    # MODIFICAÇÕES A PARTIR DAQUI

    # Curva parametrizada do jogador
    # Equação da curva - Exemplo de um círculo: x(t) = cos(t) e y(t) = sin(t).
    # Usamos 2*pi*theta para que a velocidade angular seja v (em rotações por segundo)
    # (200, 200) é o centro do círculo
    # 100 é o raio do círculo (quanto maior, mais afastado do centro da tela)
    fx = 200 + 100 * np.cos(2*np.pi*theta)
    fy = 200 - 100 * np.sin(2*np.pi*theta)

    # Posição dos pontos
    s = np.array([[fx], [fy], [1]])

    # Vetor tangente (derivada da curva)
    dfx = 100 * 2*np.pi * np.sin(2*np.pi*theta)
    dfy = 100 * 2*np.pi * np.cos(2*np.pi*theta)
    argumento = np.arctan2(dfy, dfx)  # ângulo entre vetor tangente e eixo horizontal

    # Transformação dos pontos
    # 1. Translada para o referencial
    pontos_rel = pontos_loc.copy()
    pontos_rel[0,:] -= s[0,0]
    pontos_rel[1,:] -= s[1,0]

    # 2. Rotaciona
    R = np.array([
        [np.cos(np.pi/2 - argumento), -np.sin(np.pi/2 - argumento), 0],
        [np.sin(np.pi/2 - argumento),  np.cos(np.pi/2 - argumento), 0],
        [0, 0, 1]
    ])
    pontos_rot = R @ pontos_rel

    # 3. Translada para o centro da tela
    pontos_final = pontos_rot.copy()
    pontos_final[0,:] += 200
    pontos_final[1,:] += 200

    screen.fill(BLACK)
    for p in range(pontos_loc.shape[1]):
        rect = pygame.Rect(pontos_final[0:2,p], (2, 2))
        screen.blit(pontos, rect)

    # Desenhar personagem (no centro da tela)
    rect = pygame.Rect((200,200), (10, 10))
    screen.blit(personagem, rect)

    pygame.display.update()
pygame.quit()